# Test 4

### Import

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from backtesting import Backtest
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os
from datetime import datetime
from lumibot.brokers import Alpaca
import matplotlib.pyplot as plt
from lumibot.backtesting import YahooDataBacktesting
import numpy as np
from dotenv import load_dotenv
load_dotenv()

In [2]:
file_path = "../Models/best_model.pt"

# Delete current model
if os.path.exists(file_path):
    os.remove(file_path)

### Device

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

device = "cpu"
print(f"Using device: {device}")

### Hyperparameter

In [4]:
# Model parameter
input_size = 20
output_size = 1
hidden_size = 10
num_layers = 4
dropout = 0.2

# Training parameter
batch_size = 1
num_epochs = 10
learning_rate = 0.0001
seq_size = 30

train_data_path = "../Data/train_dax_data.pkl"
test_data_path = "../Data/test_dax_data.pkl"

### LSTM Model

In [5]:
class Net(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers, dropout=0.2):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :]) 
        
        return x


### Dataloader

In [6]:
class FinanceDataset(Dataset):
    def __init__(self, input, output, seq_size):
        self.seq_size = seq_size
        
        self.inputs = input
        self.labels = output
        
    def __len__(self):
        return len(self.inputs) - self.seq_size

    def __getitem__(self, idx):
        x = self.inputs[idx:idx + self.seq_size]
        y = self.labels[idx + self.seq_size] 

        # Convert to tensors
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        return x, y

### Init

In [7]:
# Initialize model, loss function, optimizer
net = Net(input_size, output_size, hidden_size, num_layers)
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

# Data

In [ ]:
# Train
df = pd.read_pickle(train_data_path)

df = df[['^GDAXI_Open', '^GDAXI_High', '^GDAXI_Low', '^GDAXI_Close',
                                '^GDAXI_Adj Close', '^GDAXI_Volume', '^GDAXI_month', '^GDAXI_weekday',
                                'GC=F_Open', 'GC=F_High', 'GC=F_Low', 'GC=F_Close',
                                'GC=F_Adj Close', 'GC=F_Volume',
                                'BZ=F_Open', 'BZ=F_High', 'BZ=F_Low', 'BZ=F_Close',
                                'BZ=F_Adj Close', 'BZ=F_Volume',
                                'Y']]

display(df)

scaler = MinMaxScaler()
scaler_y = MinMaxScaler()

scaled_train_inputs = scaler.fit_transform(df.iloc[:, :-1].values)  
scaled_train_labels = scaler_y.fit_transform(df.iloc[:, -1].values.reshape(-1, 1))

In [ ]:
# Test
test_df = pd.read_pickle(test_data_path)

test_df = test_df[['^GDAXI_Open', '^GDAXI_High', '^GDAXI_Low', '^GDAXI_Close',
                                '^GDAXI_Adj Close', '^GDAXI_Volume', '^GDAXI_month', '^GDAXI_weekday',
                                'GC=F_Open', 'GC=F_High', 'GC=F_Low', 'GC=F_Close',
                                'GC=F_Adj Close', 'GC=F_Volume',
                                'BZ=F_Open', 'BZ=F_High', 'BZ=F_Low', 'BZ=F_Close',
                                'BZ=F_Adj Close', 'BZ=F_Volume',
                                'Y']]


display(test_df)

scaled_test_inputs = scaler.transform(test_df.iloc[:,:-1].values) 
scaled_test_labels = scaler_y.transform(test_df.iloc[:, -1].values.reshape(-1, 1))

In [10]:
# Initialize dataset and dataloader
dataset = FinanceDataset(scaled_train_inputs, scaled_train_labels, seq_size=seq_size)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

test_dataset = FinanceDataset(scaled_test_inputs, scaled_test_labels, seq_size=seq_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

### Training

In [ ]:
os.makedirs("../Models", exist_ok=True)

# Modellpfad festlegen
best_model_path = "../Models/best_model.pt"

# Modell laden, falls es bereits existiert
if os.path.exists(best_model_path):
    net.load_state_dict(torch.load(best_model_path))
    print(f"Modell erfolgreich geladen von {best_model_path}")

# Parameter für Early Stopping und Modell-Speicherung
patience = 8
best_test_loss = float('inf')
early_stopping_counter = 0

losses = []
test_loss_vals = []

# Training loop mit Early Stopping
for epoch in range(num_epochs):
    net.train()
    epoch_loss = 0
    epoch_output = []
    # Training
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = net(inputs)
        epoch_output.append(outputs.detach().numpy())
        loss = criterion(outputs.squeeze(-1), labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    epoch_loss /= len(train_loader)
    avg_train_loss = epoch_loss / len(train_loader)
    losses.append(epoch_loss)

    # Modell auswerten auf dem Testset
    net.eval()
    running_test_loss = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            test_outputs = net(inputs)
            test_loss = criterion(test_outputs.squeeze(-1), labels)
            running_test_loss += test_loss.item()

        avg_test_loss = running_test_loss / len(test_loader)
        test_loss_vals.append(avg_test_loss)

        # Check for Early Stopping und Speichern des besten Modells
        if avg_test_loss < best_test_loss:
            best_epoch = epoch
            best_test_loss = avg_test_loss
            torch.save(net.state_dict(), best_model_path)
            print(f"Bestes Modell gespeichert mit Test Loss: {best_test_loss:.4f}")
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        # Ausgabe der Verluste
        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

        # Early Stopping-Kriterium prüfen
        if early_stopping_counter >= patience:
            print(f"Early stopping nach {epoch + 1} Epochen. Test loss verbesserte sich nicht in den letzten {patience} Epochen.")
            break
    # print(f"Std Epoch Out: {np.std(epoch_output)}")
    # print(f"Mean Epoch Out: {np.mean(epoch_output)}")

# Plot der Trainings- und Testverluste
plt.figure(figsize=(10, 5))
plt.plot(losses, label='Training Loss')
plt.plot(test_loss_vals, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training und Test Loss')
plt.grid(True)
plt.show()

print(f"Training abgeschlossen. Bestes Modell gespeichert unter: {best_model_path}")
print(f"Std Training Loss: {np.std(losses)}")
print(f"Std Test Loss: {np.std(test_loss_vals)}")
print(f"Min Training Loss: {np.min(losses)}")
print(f"Min Test Loss: {np.min(test_loss_vals)}")

### Backtesting

In [ ]:
scaler = MinMaxScaler()
scaler_y = MinMaxScaler()

df = pd.read_pickle(train_data_path)
df = df[['^GDAXI_Open', '^GDAXI_High', '^GDAXI_Low', '^GDAXI_Close',
                                '^GDAXI_Adj Close', '^GDAXI_Volume', '^GDAXI_month', '^GDAXI_weekday',
                                'GC=F_Open', 'GC=F_High', 'GC=F_Low', 'GC=F_Close',
                                'GC=F_Adj Close', 'GC=F_Volume',
                                'BZ=F_Open', 'BZ=F_High', 'BZ=F_Low', 'BZ=F_Close',
                                'BZ=F_Adj Close', 'BZ=F_Volume',
                                'Y']]

display(df)

scaled_train_inputs = scaler.fit(df.iloc[:, :-1].values)  
scaled_train_labels = scaler_y.fit(df.iloc[:, -1].values.reshape(-1, 1))

model_path = "../Models/best_model.pt"

In [ ]:
import pandas as pd
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
import numpy as np

model = Net(input_size, output_size, hidden_size, num_layers)

# Load state_dict only
model.load_state_dict(torch.load(model_path)) 
model.eval()

df = pd.read_pickle(test_data_path)
df = df[['^GDAXI_Open', '^GDAXI_High', '^GDAXI_Low', '^GDAXI_Close',
                                '^GDAXI_Adj Close', '^GDAXI_Volume', '^GDAXI_month', '^GDAXI_weekday',
                                'GC=F_Open', 'GC=F_High', 'GC=F_Low', 'GC=F_Close',
                                'GC=F_Adj Close', 'GC=F_Volume',
                                'BZ=F_Open', 'BZ=F_High', 'BZ=F_Low', 'BZ=F_Close',
                                'BZ=F_Adj Close', 'BZ=F_Volume',
                                'Y']]
display(df)
scaled_test_inputs = scaler.transform(df.iloc[:, :-1].values)  
scaled_test_labels = scaler_y.transform(df.iloc[:, -1].values.reshape(-1, 1))

test_data = FinanceDataset(scaled_test_inputs, scaled_test_labels, seq_size=seq_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

all_predictions = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        out = model(inputs) 
        
        all_predictions.append(out.numpy())  
        all_labels.append(labels.numpy())

all_predictions = np.concatenate(all_predictions)
all_labels = np.concatenate(all_labels)

print(f'Predicted values: {all_predictions.flatten()}')
print(f'Actual values: {all_labels.flatten()}')

output_df = pd.DataFrame({'Predicted': all_predictions.flatten(), 'Actual': all_labels.flatten()})
display(output_df)


In [ ]:
pred = scaler_y.inverse_transform(all_predictions).reshape(-1, 1)
actual = scaler_y.inverse_transform(all_labels.reshape(-1, 1)).reshape(-1, 1)

# DataFrame erstellen
df_results = pd.DataFrame({
    "pred": pred.flatten(),
    "actual": actual.flatten(),
})

# DataFrame anzeigen
print(df_results)

In [ ]:
os.makedirs("logs", exist_ok=True)
os.makedirs("results", exist_ok=True)

test_data = pd.read_pickle(test_data_path)
train_data = pd.read_pickle(train_data_path)

test_data = test_data[['Date','^GDAXI_Open', '^GDAXI_High', '^GDAXI_Low', '^GDAXI_Close',
                                '^GDAXI_Adj Close', '^GDAXI_Volume', '^GDAXI_month', '^GDAXI_weekday',
                                'GC=F_Open', 'GC=F_High', 'GC=F_Low', 'GC=F_Close',
                                'GC=F_Adj Close', 'GC=F_Volume',
                                'BZ=F_Open', 'BZ=F_High', 'BZ=F_Low', 'BZ=F_Close',
                                'BZ=F_Adj Close', 'BZ=F_Volume',
                                'Y']]
train_data = train_data[['^GDAXI_Open', '^GDAXI_High', '^GDAXI_Low', '^GDAXI_Close',
                                '^GDAXI_Adj Close', '^GDAXI_Volume', '^GDAXI_month', '^GDAXI_weekday',
                                'GC=F_Open', 'GC=F_High', 'GC=F_Low', 'GC=F_Close',
                                'GC=F_Adj Close', 'GC=F_Volume',
                                'BZ=F_Open', 'BZ=F_High', 'BZ=F_Low', 'BZ=F_Close',
                                'BZ=F_Adj Close', 'BZ=F_Volume',
                                'Y']]

scaler = MinMaxScaler()
scaler.fit(train_data.iloc[:, :-1].values)

scaler_y = MinMaxScaler()
scaler_y.fit(train_data.iloc[:, -1].values.reshape(-1, 1))

model = Net(input_size, output_size, hidden_size, num_layers)
model_path = "../Models/best_model.pt"
model.load_state_dict(torch.load(model_path))
model.eval()

ALPACA_CREDS = {
    "API_KEY": os.getenv("ALPACA_API_KEY"),
    "API_SECRET": os.getenv("ALPACA_API_SECRET"),
    "PAPER": True,
}

# Strategy setup
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)
broker = Alpaca(ALPACA_CREDS)

strategy = Backtest(
    name="Test4-Raw Material",
    broker=broker,
    parameters={
        "symbol": "^GDAXI",
        "cash_at_risk": 0.5,
        "model": model,
        "num_prior_days": 30,
        "dataset": test_data,
        "scaler": scaler,
        "scaler_y": scaler_y,
    },
)

# Run backtest
backtest_results = strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    name="Test4-Raw Material",
    parameters={
        "symbol": "^GDAXI",
        "cash_at_risk": 0.5,
        "model": model,
        "dataset": test_data,
        "num_prior_days": 30,
        "scaler": scaler,
        "scaler_y": scaler_y,
    },
    benchmark_asset="^GDAXI",
    show_plot=True,
    show_tearsheet=True,
)

# Save results
pd.DataFrame(backtest_results).to_csv("results/backtest_results.csv.gz", index=False, compression="gzip")

print("Backtesting complete. Results saved to backtest_results.csv.gz.")

In [ ]:
# Backtesting-Funktion
def backtest_model(model, dataloader, scaler_y):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            # Vorhersage
            output = model(X_batch)
            predictions.extend(output.numpy())
            actuals.extend(y_batch.numpy())

    # Rücktransformation der Vorhersagen und tatsächlichen Werte
    predictions = scaler_y.inverse_transform(np.array(predictions).reshape(-1, 1))
    actuals = scaler_y.inverse_transform(np.array(actuals).reshape(-1, 1))

    return predictions.flatten(), actuals.flatten()

# Backtesting starten
def run_backtest(test_df, model_path, seq_size):
    # Daten vorverarbeiten
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    train_data = pd.read_pickle(train_data_path)
    train_data = train_data[['^GDAXI_Open', '^GDAXI_High', '^GDAXI_Low', '^GDAXI_Close',
                                '^GDAXI_Adj Close', '^GDAXI_Volume', '^GDAXI_month', '^GDAXI_weekday',
                                'GC=F_Open', 'GC=F_High', 'GC=F_Low', 'GC=F_Close',
                                'GC=F_Adj Close', 'GC=F_Volume',
                                'BZ=F_Open', 'BZ=F_High', 'BZ=F_Low', 'BZ=F_Close',
                                'BZ=F_Adj Close', 'BZ=F_Volume',
                                'Y']]    
    scaler_X.fit(train_data.iloc[:, 1:-1].values)
    scaler_y.fit(train_data.iloc[:, -1].values.reshape(-1, 1))

    X_test = test_df.iloc[:, 1:-1]
    y_test = test_df.iloc[:, -1]

    X_test_scaled = scaler_X.transform(X_test.values)
    y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

    # Sequenzen erstellen
    test_dataset = FinanceDataset(X_test_scaled, y_test_scaled, seq_size)
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    # Modell laden
    model = Net(input_size=input_size, output_size=output_size, hidden_size=hidden_size, num_layers=num_layers)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    # Backtesting durchführen
    predictions, actuals = backtest_model(model, test_dataloader, scaler_y)

    # Ergebnisse visualisieren
    plt.figure(figsize=(14, 7))
    plt.plot(predictions, label="Predicted", color="blue")
    plt.plot(actuals, label="Actual", color="orange")
    plt.title("Backtesting Results")
    plt.legend()
    plt.show()

    # Statistiken berechnen
    df_results = pd.DataFrame({"Actual": actuals, "Predicted": predictions})
    mse = ((df_results["Actual"] - df_results["Predicted"]) ** 2).mean()
    mae = np.abs(df_results["Actual"] - df_results["Predicted"]).mean()
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")

    return df_results

# Anwendung der Backtesting-Funktion
test_data = pd.read_pickle(test_data_path)  # Testdatensatz laden
test_data = test_data[['^GDAXI_Open', '^GDAXI_High', '^GDAXI_Low', '^GDAXI_Close',
                                '^GDAXI_Adj Close', '^GDAXI_Volume', '^GDAXI_month', '^GDAXI_weekday',
                                'GC=F_Open', 'GC=F_High', 'GC=F_Low', 'GC=F_Close',
                                'GC=F_Adj Close', 'GC=F_Volume',
                                'BZ=F_Open', 'BZ=F_High', 'BZ=F_Low', 'BZ=F_Close',
                                'BZ=F_Adj Close', 'BZ=F_Volume',
                                'Y']]
model_path = "../Models/best_model.pt"  # Pfad zum gespeicherten Modell

results = run_backtest(test_data, model_path, seq_size)